### ***최근접 지하철역***
1 .CSV 로드 (금별맥주_정제완료.csv)
2. 좌표 컬럼 표준화 (위도 → lat, 경도 → lng)
3. 좌표 없는 경우 처리 (nearest_station = "좌표 정보 없음")
4. 좌표 있는 경우 → 최근접 지하철역과 거리 계산 (nearest_station, distance_m)
5. 최종 CSV 저장 (금별_최근접_지하철역.csv)

In [96]:
# 라이브러리 임포트
import pandas as pd
from math import radians, cos, sin, asin, sqrt
import os

In [97]:
# 경로 설정
OUTPUT_DIR = r"C:\Users\user\Desktop\데이터분석\출력"
os.makedirs(OUTPUT_DIR, exist_ok=True)

ORIGINAL_PATHS = {
    "역전": r"C:\Users\user\Desktop\데이터분석\역전할머니_정제완료.csv",
    "금별": r"C:\Users\user\Desktop\데이터분석\금별맥주_정제완료.csv"
}

OUTPUT_PATHS = {
    "역전": os.path.join(OUTPUT_DIR, "역전_최근접_지하철역.csv"),
    "금별": os.path.join(OUTPUT_DIR, "금별_최근접_지하철역.csv")
}

STATION_PATH = r"C:\Users\user\Desktop\데이터분석\서울시 역사마스터 정보.csv"

In [98]:
# CSV 안전 로드
# =============================
# =============================
def safe_read_csv(path):
    try:
        df = pd.read_csv(path, encoding="utf-8-sig")
    except UnicodeDecodeError:
        df = pd.read_csv(path, encoding="cp949")
    df.columns = df.columns.str.strip()
    return df

In [99]:
# 서울/경기 지하철 역사 마스터 로드
# =============================
stations = safe_read_csv(STATION_PATH)
stations = stations.rename(columns={"지하철역명": "station_name", "위도": "lat", "경도": "lng"})
stations = stations[["station_name", "lat", "lng"]].dropna()

In [100]:
# 거리 계산 함수
# =============================
def haversine(lat1, lng1, lat2, lng2):
    lat1, lng1, lat2, lng2 = map(radians, [lat1, lng1, lat2, lng2])
    dlat = lat2 - lat1
    dlng = lng2 - lng1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlng/2)**2
    c = 2 * asin(sqrt(a))
    return 6371000 * c  # meter

In [104]:
# 최근접 지하철 계산
# =============================
def find_nearest_subway(lat, lng, stations_df):
    distances = stations_df.apply(lambda r: haversine(lat, lng, r["lat"], r["lng"]), axis=1)
    idx = distances.idxmin()
    station_name = stations_df.loc[idx, "station_name"]
    # 항상 '역' 붙이기
    if not station_name.endswith("역"):
        station_name += "역"
    return station_name

In [105]:
# 브랜드별 처리 함수
# =============================
def process_brand(input_csv, output_csv):
    print(f"\n▶ 처리 시작: {input_csv}")
    
    df = safe_read_csv(input_csv)
    
    # 컬럼 표준화
    if '위도' in df.columns and '경도' in df.columns:
        df = df.rename(columns={"위도": "lat", "경도": "lng"})
    
    # 좌표 없는 경우 처리
    no_coord = df[df[["lat","lng"]].isna().any(axis=1)]
    df.loc[no_coord.index, "nearest_station"] = "좌표 정보 없음"
    
    # 좌표 있는 경우 → 최근접 역 계산
    has_coord = df.dropna(subset=["lat","lng"])
    if not has_coord.empty:
        result = has_coord.apply(lambda r: find_nearest_subway(r["lat"], r["lng"], stations), axis=1)
        df.loc[has_coord.index, "nearest_station"] = result.values
    
    # 필요한 컬럼만 선택
    output_df = df[["place_id", "지점명", "lat", "lng", "nearest_station"]]
    
    # 최종 CSV 저장
    output_df.to_csv(output_csv, index=False, encoding="utf-8-sig")
    print(f"✔ 저장 완료: {output_csv}")
    print(output_df.head(10))  # 상위 10개 샘플 출력

In [106]:
# 최종 실행 (각 브랜드 별도 저장)
# =============================
process_brand(ORIGINAL_PATHS["역전"], OUTPUT_PATHS["역전"])
process_brand(ORIGINAL_PATHS["금별"], OUTPUT_PATHS["금별"])


▶ 처리 시작: C:\Users\user\Desktop\데이터분석\역전할머니_정제완료.csv
✔ 저장 완료: C:\Users\user\Desktop\데이터분석\출력\역전_최근접_지하철역.csv
     place_id     지점명        lat         lng nearest_station
0  1868359886     구청점  37.552714  126.852178             가양역
1  1323812622    명지대점  37.579687  126.925270       증산(명지대앞)역
2  1311695977  서울신풍역점  37.498765  126.908128             신풍역
3  1650551571   서울상봉점  37.594237  127.088287    상봉(시외버스터미널)역
4  1404198758     오류점  37.497175  126.844202            오류동역
5  1054587722  서울방배역점  37.482051  126.996653             방배역
6  1573115649     NaN  37.591610  127.051700             회기역
7  1407805860     NaN  37.619273  127.044889            돌곶이역
8  1420076532    화곡역점  37.540438  126.835106             화곡역
9  1337289813    삼거리점  37.499175  126.927951         신대방삼거리역

▶ 처리 시작: C:\Users\user\Desktop\데이터분석\금별맥주_정제완료.csv
✔ 저장 완료: C:\Users\user\Desktop\데이터분석\출력\금별_최근접_지하철역.csv
     place_id     지점명        lat         lng nearest_station
0  1113384749  동양미래대점  37.500095  126.867221       